<a href="https://colab.research.google.com/github/tarang1998/ML-AI-DL/blob/main/gen-ai/vectorDB/pinecone/Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [2]:
!pip install openai
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.5 MB/s eta 0:00:00


In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [4]:
!mkdir pdfs
# Upload PDFS in this directory

In [5]:
!gdown 1aLMpgxJoBHJfqGKMA82M1VK-tW9rXLIT -O pdfs/tarangResume.pdf



Downloading...
From: https://drive.google.com/uc?id=1aLMpgxJoBHJfqGKMA82M1VK-tW9rXLIT
To: /content/pdfs/tarangResume.pdf
100% 130k/130k [00:00<00:00, 29.5MB/s]


In [6]:
# Extract text data from the pdf

loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()
data

[Document(metadata={'producer': 'Skia/PDF m119 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Resume - Tarang Nair', 'source': 'pdfs/tarangResume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='TARANGNAIRArlington,Virginia| +1571-385-5625| tarangnair98@gmail.com| linkedin.com/in/tarangnair1998| github.com/tarang1998|tarangnair.com\nEDUCATION-UniversityOfMaryland,CollegePark Maryland,USMastersOfEngineeringinSoftwareEngineering Expected2025\n-VidyalankarInstituteOfTechnology,MumbaiUniversity Mumbai,IndiaBachelorsOfEngineeringinComputerEngineering,C.G.P.A: 8.41/10 July2016-July2020\nSKILLS● TechnicalSkills:HTML,CSS,JavaScript,ReactJS,TypeScript,SQL,Python,C,C++,Java,Dart,GoogleCloudPlatforms-Firebase(NoSQL)| CloudSpanner(SQL)| BigQuery| CloudFunctions,Git,Flutter,Excel,GoogleOfficeSuite.● Languages:English(Advanced),Hindi(Advanced),Malayalam(Intermediate)\nWORKEXPERIENCEXeroApps Mumbai,India(Remote)SoftwareDevelopmentEngineer Dec2022-July2023

In [7]:
# Split the extracted data into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)
text_chunks

[Document(metadata={'producer': 'Skia/PDF m119 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Resume - Tarang Nair', 'source': 'pdfs/tarangResume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='TARANGNAIRArlington,Virginia| +1571-385-5625| tarangnair98@gmail.com| linkedin.com/in/tarangnair1998| github.com/tarang1998|tarangnair.com\nEDUCATION-UniversityOfMaryland,CollegePark Maryland,USMastersOfEngineeringinSoftwareEngineering Expected2025\n-VidyalankarInstituteOfTechnology,MumbaiUniversity Mumbai,IndiaBachelorsOfEngineeringinComputerEngineering,C.G.P.A: 8.41/10 July2016-July2020'),
 Document(metadata={'producer': 'Skia/PDF m119 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Resume - Tarang Nair', 'source': 'pdfs/tarangResume.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='SKILLS● TechnicalSkills:HTML,CSS,JavaScript,ReactJS,TypeScript,SQL,Python,C,C++,Java,Dart,GoogleCloudPlatforms-Firebase(NoS

In [8]:
# Download the openAI Embeddings
import os
import openai
import getpass

openai.api_key = getpass.getpass(prompt='Enter OpenAI API key:')
os.environ['OPENAI_API_KEY'] = openai.api_key

Enter OpenAI API key:··········


In [10]:
embeddings = OpenAIEmbeddings()
result = embeddings.embed_query("My name is Tarang")
len(result) # The length of the embeddings would be same for any sentence

1536

In [11]:
# Initializing PineCone
os.environ['PINECONE_API_KEY'] = getpass.getpass(prompt='Enter Pinecone API key:')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')


Enter Pinecone API key:··········


In [13]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )
index_name = "test" # put in the name of your pinecone index here


In [20]:
from langchain_community.vectorstores import Pinecone


In [21]:
# Create embeddings from the chunks
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [24]:
# Loading the created index from pinecone with all the records
docsearch = Pinecone.from_existing_index(index_name, embeddings)
docsearch

In [26]:
# Similarity Search
query = "Is Tarang Nair a good engineer"

In [27]:
docs = docsearch.similarity_search(query, k=3)
docs

[Document(metadata={}, page_content='TARANGNAIRArlington,Virginia| +1571-385-5625| tarangnair98@gmail.com| linkedin.com/in/tarangnair1998| github.com/tarang1998|tarangnair.com\nEDUCATION-UniversityOfMaryland,CollegePark Maryland,USMastersOfEngineeringinSoftwareEngineering Expected2025\n-VidyalankarInstituteOfTechnology,MumbaiUniversity Mumbai,IndiaBachelorsOfEngineeringinComputerEngineering,C.G.P.A: 8.41/10 July2016-July2020'),
 Document(metadata={}, page_content='-DeveloperPortfolioWebsite(tarangnair.com) Mumbai,IndiaSoftwareDeveloper Mar2020-May2020● DevelopedawebapplicationusingHTML,JavaScript,Bootstrap,ReactJSanditslibrariestodisplaytheindividualsskills,education,projectsandotherinfotorecruiters\nACTIVITIESSmartIndiaHackathon March2019,Chennai● Representedourcollegeina36hournationallevelhackathonconductedatSathyabamaUniversity,Chennaiwhereourteamcameupwithasolutiontotheassignedproblemstatement.'),
 Document(metadata={}, page_content='SKILLS● TechnicalSkills:HTML,CSS,JavaScript,Reac

In [28]:
# Create a LLM Model Wrapper
llm = OpenAI()
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


<ipython-input-28-27f9c32cae7a>:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()


In [29]:
qa.run(query)

<ipython-input-29-fbc3ad05536a>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


' Based on the information provided, it appears that Tarang Nair has a strong educational background in software engineering and has gained experience in various technical skills through internships and personal projects. However, without further information or evidence, it is not possible to determine if Tarang Nair is a good engineer. '

In [30]:
query = "Tell me about Tarang Nair's experience"
qa.run(query)

'\nBased on the context provided, Tarang Nair appears to be a software developer from Mumbai, India. He has experience in developing web applications using various technologies such as HTML, JavaScript, Bootstrap, and ReactJS. He has also participated in a national level hackathon and represented his college. Tarang has a Masters of Engineering in Software Engineering from the University of Maryland and a Bachelors of Engineering in Computer Engineering from Vidyalankar Institute of Technology. He has also worked as a Java Software Developer intern at Godrej InfoTech Limited and has experience in programming web applications using Spring MVC Java Framework. Additionally, Tarang has worked on projects such as KiranaFast and EaseMyStore as a software developer and has also developed an augmented reality mobile application called SnappyFoodMenu. He has expertise in Unity3D and has designed features to enhance the user experience for restaurant customers. '

In [31]:
import sys

In [32]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Input Prompt: Does Tarang know ML ? 


<ipython-input-32-c213a85929f6>:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'query': user_input})


Answer:  Yes, based on the context provided, Tarang has experience with ML as he developed an email classifier using ML algorithms in a hackathon and also worked on programming web applications using the Spring MVC Java framework.
Input Prompt: Based on this resume, would he be employable ?
Answer: Yes, it appears that the individual has a strong technical background and has experience working on various projects and internships. They also have experience with popular software and tools, which could make them a valuable asset to potential employers. However, their work experience and project experience may vary and ultimately, it would depend on the employer's specific requirements and preferences.
Input Prompt: exit
Exiting


SystemExit: 

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
